「PyTorch入門  4. モデル構築」
===============================================================
【原題】BUILD THE NEURAL NETWORK

【原著】
[Suraj Subramanian](https://github.com/suraj813)、[Seth Juarez](https://github.com/sethjuarez/) 、[Cassie Breviu](https://github.com/cassieview/) 、[Dmitry Soshnikov](https://soshnikov.com/)、[Ari Bornstein](https://github.com/aribornstein/) 


【元URL】https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

【翻訳】電通国際情報サービスISID AIトランスフォーメーションセンター　小川 雄太郎

【日付】2021年03月18日

【チュトーリアル概要】

本チュートリアルでは、PyTorchでニューラルネットワークのモデルを構築する方法について解説します

---


ニューラルネットワークモデルの作り方
===================

ニューラルネットワークは、レイヤー（もしくはモジュール）と呼ばれるデータ操作の固まりで構成されています。

[`torch.nn`](https://pytorch.org/docs/stable/nn.html)で用意されているクラス、関数は、独自のニューラルネットワークを構築するために必要な要素を網羅しています。



PyTorchの全てのモジュールは、[`nn.Module`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)を継承しています。


そしてニューラルネットワークは、モジュール自体が他のモジュール（レイヤー）から構成されています。

<br>

この入れ子構造により、複雑なアーキテクチャを容易に構築・管理することができます。

以下で、FashionMNIST datasetの画像データをクラス分類するネットワークモデルを構築します。




In [1]:
%load_ext lab_black

In [2]:
%matplotlib inline

In [3]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

---

訓練に使用するデバイス
-----------------------
可能であれば高速に訓練を行うためにも、GPU等を利用したいところです。

GPUが使用可能かどうかは、[`torch.cuda`](https://pytorch.org/docs/stable/notes/cuda.html)の結果から判断できます。

GPUが使用できない場合は、CPUで訓練を行います。



In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cpu device


---



クラスの定義
-------------------------
``nn.Module``を継承し、独自のネットワークモデルを定義し、その後ネットワークのレイヤーを ``__init__``で初期化します。

``nn.Module`` を継承した全モジュールは、入力データの順伝搬関数である``forward``関数を持ちます。





In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

``NeuralNetwork``クラスのインスタンスを作成し、変数``device``上に移動させます。

以下でネットワークの構造を出力し確認します。



In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


モデルを利用するには、入力データを与える必要があります。

入力データをモデルに投入すると、``forward``関数で処理されるとともに、いくつかの[`background operations`](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866)が実行されます。

そのため、 ``model.forward()`` と記載して入力データを処理しないように注意してください。



今回のモデルに入力を与えると、各クラスの生の予測値を含む10次元のテンソルが返されます。

``nn.Softmax``モジュールにこの出力結果を与えることで、入力データが各クラスに属する確率の予測値を求めることができます。





In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([0])


--------------




モデルレイヤー
-------------------------

FashionMNISTモデルを各レイヤーレベルで確認しましょう。

サイズ28x28の3枚の画像からなるミニバッチのサンプルを用意し、このミニバッチをネットワークに入力し、各処理による変化を確認していきます。



In [8]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


**nn.Flatten**

[`nn.Flatten`](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)レイヤーで、2次元（28x28）の画像を、1次元の784ピクセルの値へと変換します。

ミニバッチの0次元目は、サンプル番号を示す次元で、この次元は`nn.Flatten`を通しても変化しません（1次元目以降がFlattenされます）。



In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


**nn.Linear** 


[`linear layer`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)は、線形変換を施します。

`linear layer`は重みとバイアスのパラメータを保持しています。





In [10]:
layer1 = nn.Linear(in_features=28 * 28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


**nn.ReLU**

非線形な活性化関数は、ニューラルネットワークの入力と出力の間にある、複雑な関係性を表現するために重要な要素です。

これらの活性化関数は線形変換のあとに、非線形性を加え、ニューラルネットワークの表現力を向上させる役割をします。

<br>

今回のモデルでは[`nn.ReLU`](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)をlinear layerの間に用意しました。

なお、非線形活性化関数には他にも様々な種類があります。



In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.2828, -0.5616,  0.0528,  0.1397, -0.0168, -0.5177, -0.4881, -0.1613,
          0.4703, -0.1115, -0.0919,  0.6296,  0.5314, -0.1182,  0.1555,  0.1564,
         -0.0394, -0.1634,  0.5461,  0.0099],
        [ 0.3341, -0.6381,  0.0701, -0.1267,  0.1408, -0.4912, -0.4336, -0.4294,
          0.2513,  0.4710, -0.4369,  0.1852,  0.4607, -0.1675,  0.1243,  0.2947,
          0.2342, -0.2011,  0.6222, -0.4276],
        [ 0.0210, -0.4948,  0.0339,  0.0770,  0.2622, -0.7262, -0.2526, -0.7636,
          0.1175, -0.2621, -0.3155,  0.2871,  0.6108, -0.3881,  0.1238,  0.4461,
          0.0707, -0.1718,  0.4002,  0.2049]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.2828, 0.0000, 0.0528, 0.1397, 0.0000, 0.0000, 0.0000, 0.0000, 0.4703,
         0.0000, 0.0000, 0.6296, 0.5314, 0.0000, 0.1555, 0.1564, 0.0000, 0.0000,
         0.5461, 0.0099],
        [0.3341, 0.0000, 0.0701, 0.0000, 0.1408, 0.0000, 0.0000, 0.0000, 0.2513,
         0.4710, 0.0000, 0.1852, 0.4607, 0.0000, 0.124

**nn.Sequential**

[``nn.Sequential``](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)は、モジュールを順番に格納する箱のような要素です。

入力データは``nn.Sequential``に定義された順番に各モジュールを伝搬します。

<br>

例えば以下の実装例のように、``seq_modules``と名付けた、複数のモジュールを束ねたモジュールを簡単に構築できます。




In [12]:
seq_modules = nn.Sequential(flatten, layer1, nn.ReLU(), nn.Linear(20, 10))
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

**nn.Softmax**

ニューラルネットワークの最後のlinear layerは`logits` [- ∞, ∞] を出力します。


この`logits`は[`nn.Softmax`](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)モジュールへと渡されます。


その結果、採取的な値は[0, 1]の範囲となり、これは各クラスである確率を示します。


``dim``パラメータは次元を示しており、`dim=1`の次元で和を求めると確率の総和なので1になります。




In [13]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

---




モデルパラメータ
-------------------------

ニューラルネットワークを構成する多くのモジュールは、おのおのパラメータを保持しています。

例えば、重みやバイアスです。これらの値が訓練時に最適化されます。




``nn.Module`` を継承することで、モデルオブジェクト内で定義されたすべてのフィールドが自動的に追跡でき、``parameters()`` や ``named_parameters()`` メソッドを使って、モデルの各レイヤーのすべてのパラメータにアクセスできるようになります。

以下にfor文を用いて、各パラメータを処理し、そのサイズと値を表示します。



In [14]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0247, -0.0044, -0.0301,  ...,  0.0277, -0.0060,  0.0030],
        [ 0.0277,  0.0109, -0.0351,  ..., -0.0151, -0.0193, -0.0127]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0180,  0.0257], grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0168,  0.0094,  0.0121,  ...,  0.0023, -0.0140,  0.0076],
        [ 0.0261, -0.0128,  0.0288,  ..., -0.0162, -0.0053,  0.0254]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_

--------------




さらなる詳細
--------------
以下のページも参考ください。

- [`torch.nn API`](https://pytorch.org/docs/stable/nn.html)



以上。